# Run these Regardless

In [ ]:
!pip install -q gpt-2-simple

In [ ]:
import re
import pandas as pd
from google.colab import drive
#from nltk.translate import bleu

%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
#from rouge_score import rouge_scorer
import tensorflow as tf
from tensorflow.python.framework import ops

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Fine-tuning GPT-2 on a Custom Dataset

## Setup

Make sure you enable a GPU or TPU in the runtime  
Runtime -> Change Runtime -> Hardware Accelerator 

Download GPT-2 models.  
When fine-tuning on a single GPU, only the 124M and 335M size models can be used.

In [ ]:
model_size = '124M'
gpt2.download_gpt2(model_name=model_size)

Fetching checkpoint: 1.05Mit [00:00, 303Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.10Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 422Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:17, 28.0Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 490Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.99Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.94Mit/s]


Set variables

In [ ]:
# custom_dataset.txt should be in the drive without being inside any folder
file_name = "custom_dataset.txt"
run_name = 'fine_tuning_run_1'

If your custom data is stored in your G-Drive, mount your drive and you can copy the data to Colab with the code below.  
Alternatively, you can upload your dataset directly to Colab using the Colab "Files" menu on the left (not the "File" menu above).  
Training examples in the dataset file should be separated with a blank line.

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Perform fine-tuning

In [ ]:
# Run this when you want to save the model to google drive
gpt2.copy_checkpoint_to_gdrive(run_name)

In [ ]:
#If we want to train for first time
batch_text = open("batch_text.txt", 'w')
counter = 1
with open("custom_dataset.txt", 'r') as f:
  for line in f:
    if batch_text.closed:
      batch_text = open("batch_text.txt", 'w')
    batch_text.write(line)
    if counter == 200: 
      ops.reset_default_graph()
      sess = gpt2.start_tf_sess()
      gpt2.finetune(sess,
                    dataset="batch_text.txt",
                    model_name=model_size,
                    steps=100,
                    restore_from="fresh",
                    run_name = run_name,
                    print_every=1,
                    sample_every=20,
                    save_every=50,
                    reuse=False)
      batch_text.close()
    counter += 1

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


Loading dataset...


100%|██████████| 1/1 [02:51<00:00, 171.84s/it]


dataset has 26319678 tokens
Training...
[1 | 124.30] loss=3.48 avg=3.48
[2 | 240.94] loss=3.38 avg=3.43
[3 | 357.79] loss=3.40 avg=3.42
[4 | 472.93] loss=3.47 avg=3.43
[5 | 595.65] loss=3.45 avg=3.44
[6 | 711.97] loss=3.35 avg=3.42
[7 | 827.95] loss=3.27 avg=3.40
[8 | 942.84] loss=3.24 avg=3.38
[9 | 1057.87] loss=3.24 avg=3.36
[10 | 1176.94] loss=3.42 avg=3.37
[11 | 1292.97] loss=3.13 avg=3.35
[12 | 1408.72] loss=2.84 avg=3.30
[13 | 1524.81] loss=3.08 avg=3.28
[14 | 1640.81] loss=3.25 avg=3.28
[15 | 1758.71] loss=2.97 avg=3.26
[16 | 1879.18] loss=3.00 avg=3.24
[17 | 1996.34] loss=2.85 avg=3.22
[18 | 2112.95] loss=3.50 avg=3.23
[19 | 2229.54] loss=3.34 avg=3.24
[20 | 2346.56] loss=2.78 avg=3.21
======== SAMPLE 1 ========
�-BOT. The following is a picture of it in the French style or an illustration. From the English, an expression may be read--"The present is the present, there will not be another; nothing will be the present: the present is the present, there will be another; a world o

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
# if you want to continue training a model after already trained it, you should
# use restore = "latest" and reuse_ = tf.compat.v1.AUTO_REUSE
batch_text = open("batch_text.txt", 'w')
counter = 1
with open("custom_dataset.txt", 'r') as f:
  for line in f:
    if batch_text.closed:
      batch_text = open("batch_text.txt", 'w')
    batch_text.write(line)
    if counter == 200: 
      ops.reset_default_graph()
      sess = gpt2.start_tf_sess()
      gpt2.finetune(sess,
                    dataset="batch_text.txt",
                    model_name=model_size,
                    steps=100,
                    restore_from="latest",
                    run_name = run_name,
                    print_every=1,
                    sample_every=20,
                    save_every=50,
                    reuse=tf.compat.v1.AUTO_REUSE)
      batch_text.close()
    counter += 1


End of training

# Load from drive

In [ ]:
#This will be the model name we want to load
run_name = 'fine_tuning_run_1'
print(run_name)

fine_tuning_run_1


In [ ]:
# gpt2 should be downloaded
model_size = '124M'
gpt2.download_gpt2(model_name=model_size)

Fetching checkpoint: 1.05Mit [00:00, 483Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.90Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 606Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:11, 44.5Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 249Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 8.61Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 7.32Mit/s]


In [ ]:
# Copy the model from G-Drive if it wasn't trained in this Colab session.
from google.colab import drive
drive.mount('/content/drive')
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the model.
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

Loading checkpoint checkpoint/fine_tuning_run_1/model-1900
INFO:tensorflow:Restoring parameters from checkpoint/fine_tuning_run_1/model-1900


## Continue training from drive

In [ ]:
# run this only if you want to continue training the checkpoint you loaded two cells above
file_name = "custom_dataset.txt"
run_name = 'fine_tuning_run_1'
gpt2.mount_gdrive()
gpt2.copy_file_from_gdrive(file_name)
batch_text = open("batch_text.txt", 'w')
counter = 1
with open("custom_dataset.txt", 'r') as f:
  for line in f:
    if batch_text.closed:
      batch_text = open("batch_text.txt", 'w')
    batch_text.write(line)

    # we only trained the 200 top stories due to hardware limitations
    if counter == 200: 
      ops.reset_default_graph()
      sess = gpt2.start_tf_sess()
      gpt2.finetune(sess,
                    dataset="batch_text.txt",
                    model_name=model_size,
                    steps=100,
                    restore_from="latest",
                    run_name = run_name,
                    print_every=1,
                    sample_every=20,
                    save_every=50,
                    reuse=tf.compat.v1.AUTO_REUSE)
      batch_text.close()
    counter += 1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/fine_tuning_run_1/model-1900
INFO:tensorflow:Restoring parameters from checkpoint/fine_tuning_run_1/model-1900
Loading dataset...


100%|██████████| 1/1 [01:52<00:00, 112.48s/it]


dataset has 26319678 tokens
Training...
Saving checkpoint/fine_tuning_run_1/model-1900
======== SAMPLE 1 ========
 my position, as he was one of the most trusted men in our society,—"I must have you to confess that it would, if I could come to you. You may trust me to-night, and you may go to bed, if your soul wants it, without being at home,—I promise you it will not be so without your consent. I should be glad to take refuge without danger from all the world,—there was always a safe place, and you would not have an enemy. But I will not go to sleep, for you know very well that you know the world, for that you will be safe from all danger. That is to tell you the whole truth,—I should be happy to go, if you would give me the strength I desire. I have the heart to do it so,—but I would not do it so much without your consent...." Then there was a sudden, rapid, and violent stir in the room. Mr. and Mrs. Rouncewell came out of the shadows, and they rushed out of the room together. It was

# Unconditional generation

In [ ]:
gpt2.generate(sess, run_name=run_name, temperature=.9, length=200, prefix=None, top_k=20, top_p = 0.95, nsamples=1)

had the differentials of that theory that, while being finally in favour of freedom of speech, it would be unjust to refuse the offer to which he was, indeed, to give credit. The rights of freedom were only justly understood by those who agreed to it and whose own forms of protection could be most easily found by those who had not. At last, as to liberty, it must be confessed that the denial was very necessary. Mr. Morland had asked him himself, and his friends, to make a exact list of all the acts which had been proposed to him; and to no purpose had he now any other of those principles against freedom. To this he was obliged to add two additional provisions, that he could now forbid the mention of the name of an individual, and he must have a right in connection with this that he could so firmly denominate and describe his own case to his own and to the justice of the district court. This second check was concluded by a declaration


# Conditional generation (give the model an input prompt)

In [ ]:
input_prompt = "Once upon a time"

In [ ]:
gpt2.generate(sess, run_name=run_name, temperature=.9, length=200, prefix=input_prompt, top_k=20, top_p = 0.95, nsamples=5)

Once upon a time does not come to speak to her pain or anger, or to give her the knowledge that she loves this man, that she must not have understood him, or that she will not at all see him at once. O, my dear My God, do not at once perchance, in this world a notice come to my eyes, or to my heart, that I am befogged, or that my reward will be completely wiped off my sleeve. But this is not all, all that comes to my lips; I say that in this present world I am like the sailor on board a ship of the Japanese government. These people will wear me into their bosoms, and they will take me off to beheaded. They will dress me in unsealed outer garments, and I will be devoured as if I had been a slave. And when I have sat to this they will utter those last words of life and death: “He is the soul of the world, that I have
Once upon a time, in the light of rising and going therefrom, the lorry comes over hill and road, not dimly lit from the road. There seems to be some difference between the 

# Testing

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate import bleu

In [ ]:
def listToString(s): 
    str1 = "" 
    for ele in s: 
        str1 += ele  
    return str1 

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate import bleu

# here we calculate the BLUE and Rouge score for the generated text
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
with open("test_txt.txt", 'r') as testset:
  counter = 0 
  for line in testset:
    # we calculated only for the first 5 books and took the avg score since it was a time demanding task
    if counter < 5:
      print("########")
      print("BOOK ", counter + 1)
      print("########")
      text_length = 70
      prefix_length = 10
      phrase = line
      phrase1 = phrase.split()
      phrase1 = phrase1[:prefix_length]
      result_gpt2 = ((gpt2.generate(sess, run_name=run_name, temperature=.9, length=text_length, prefix=" ".join(phrase1), top_k=20, top_p= 0.95, nsamples=1, return_as_list= True)))

      string1 = " ".join(phrase.split()[prefix_length:len(listToString(result_gpt2).split())]) # reference text
      string2 = " ".join(listToString(result_gpt2).split()[prefix_length:text_length+prefix_length]) # generated text

      # calculate scores
      accuracy = bleu([string1.split()], string2.split(), (1,))
      scores = scorer.score(string1, string2)

      # print the results
      print("Input /////// ", " ".join(phrase1), " ///////")
      print("Ground truth: |", string1)
      print("Generated:    |", string2)
      print("BLUE score: ", accuracy)
      print("----------")
      print("rouge1 score: ", scores['rouge1'])
      print("----------")
      print("rougeL score: ", scores['rougeL'])
      print("----------")
      print("")
    counter += 1

########
BOOK  1
########
Input ///////  clustered round the fire, by lamplight. "Well! I'm very glad  ///////
Ground truth: | to hear it," said Scrooge's nephew, "because I haven't great faith in these young housekeepers. What do you say, Topper?" Topper had clearly got his eye upon one of Scrooge's niece's sisters, for he answered that a bachelor was a wretched outcast, who had no
Generated:    | of that,â said the cadet; "even at that moment all I ask is that you do what you have to do." He readily gave the answer, saying: âI canât bear another moment to mention that, so I wish to give your father a signal for
BLUE score:  0.2
----------
rouge1 score:  Score(precision=0.30434782608695654, recall=0.2857142857142857, fmeasure=0.29473684210526313)
----------
rougeL score:  Score(precision=0.17391304347826086, recall=0.16326530612244897, fmeasure=0.16842105263157894)
----------

########
BOOK  2
########
Input ///////  of seeing great houses; after going over so many, she  ///////